In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

# Ustawienia
client_id = 'dfb4f42769e24d8a9870f81c6b660989'
client_secret = '1b41ba7464f74aee88f77303221fb4b7'
redirect_uri = 'http://localhost:8888/callback/'  # Zdefiniuj w ustawieniach aplikacji Spotify

# Uzyskaj token dostępu
sp_oauth = SpotifyOAuth(client_id, client_secret, redirect_uri, scope='user-library-read')
token_info = sp_oauth.get_access_token()
token = token_info['access_token']

# Utwórz obiekt Spotify
sp = spotipy.Spotify(auth=token)

# Pobierz dane z API Spotify
results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(f"{idx + 1}: {track['name']} - {track['artists'][0]['name']}")


C:\Users\agaal\AppData\Local\Temp\ipykernel_27248\3105807315.py:11: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token()


1: Snowman - Sia
2: Astronomy - Conan Gray
3: lowkey - NIKI
4: Cupid - FIFTY FIFTY
5: The Painted On The Moonlight - MIYEON
6: snowfall - Øneheart
7: Perfect Night - LE SSERAFIM
8: En dag när du blir stor - Benjamin Ingrosso
9: Zu dir - LEA
10: Letters - Maximillian
11: Rétine - Amir
12: Prada - cassö
13: Jesień – Tańcuj - L.U.C.
14: Bom Bom Bom - Roy Kim
15: Caramelldansen - Caramella Girls
16: Baddie - IVE
17: 행복 (MIC SWG LIVE VER) - OVAN
18: Blank Space (Taylor's Version) - Taylor Swift
19: Style (Taylor's Version) - Taylor Swift
20: Things We Took For Granted - Lee Juck
